In [ ]:
import os
import cx_Oracle
import requests
from bs4 import BeautifulSoup

os.putenv('NLS_LANG', '.UTF8')

conn = cx_Oracle.connect('bbaroni', 'bbaroni', 'localhost:1521/xe')
cursor = conn.cursor()

# SQL 구문
sql_str = "select * from recipe_data1_tbl"

# SQL 실행
cursor.execute(sql_str)

data = []

for i in range(1, 2):
    response = requests.get(f"https://www.10000recipe.com/recipe/list.html?q=%EC%95%94%EC%98%88%EB%B0%A9&order=reco&page={i}")
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select(".common_sp_list_li")

    for item in items:
        id = item.select(".common_sp_thumb a")[0]['href'].split('/')[2]
        site = "https://www.10000recipe.com/recipe/" + id
        response2 = requests.get(site)
        html2 = response2.text
        soup2 = BeautifulSoup(html2, 'html.parser')

        img_url = soup2.select_one(".centeredcrop img")['src']
        name2_element = soup2.select_one("[class='view2_summary st3'] h3")
        name2 = name2_element.text if name2_element else "No name found"
        ingredients = soup2.select("[class='ingre_list_name'] a")
        ingredient_list = [ingredient.text.strip() for ingredient in ingredients]
        ingredient_quantity = soup2.select("[class='ingre_list_ea']")
        ingredient_quantity_list = [quantity.text.strip() for quantity in ingredient_quantity]
        description = soup2.select("[class='media-body'] p")
        description_text = [desc.text.strip() for desc in description]
        data.append({
            "img": img_url,
            "name": name2,
            "ingredients": ingredient_list,
            "quantity": ingredient_quantity_list,
            "description": description_text
        })

for recipe in data:
    print("사진:", recipe["img"])
    print("이름:", recipe["name"])
    print("재료:", recipe["ingredients"])
    print("양:", recipe["quantity"])
    print("조리법:", recipe["description"])
    print()

    cursor.execute("insert into recipe_data1_tbl (num,img,name, ingredients, quantity, description) values (food_seq.nextval, :img, :name, :ingredients, :quantity, :description)",
                   img=recipe["img"],
                   name=recipe["name"],
                   ingredients=', '.join(recipe["ingredients"]),
                   quantity=', '.join(recipe["quantity"]),
                   description='\n'.join(recipe["description"]))
    conn.commit()

# Close cursor and connection
cursor.close()
conn.close()